In [1]:
import torch
from torch import nn
from torch.optim import Adam
import matplotlib.pyplot as plt
import numpy as np 
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from Buffers.ExperienceReplayBuffer import ExperienceReplay

<center> <h1> Constants 

In [2]:
SEED = 13 
GAMMA = 0.995
LR = 0.01
STEPS = 1000
BATCH_SIZE = 64
EPISODES = 1000

In [3]:
env = gym.make("Walker2d-v5",render_mode = "rgb_array")
RecordVideo(env,"../Results/PolicyBased",lambda x: x%25 == 0 and x != 0 , fps=15 )
actionNum = env.action_space.shape[0] 
stateNum = env.observation_space.shape[0]
print((actionNum,stateNum))

(6, 17)


c:\Users\pooya\miniconda3\envs\ai\lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at c:\Users\pooya\Projects\Programming\Artificial intelligence\Reinforcement Learning\Reinforement Learning\Results\PolicyBased folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [4]:
import time
env.reset()
for steps in range(1):
    action = env.action_space.sample()
    _,rewards,terminated,truncated,_=env.step(action)
    print(rewards)
    time.sleep(0.1)
env.close()

0.7509056997256911


In [5]:
class Critic(nn.Module):
    def __init__(self,stateNum,h1,h2,h3,actionNum):
        super().__init__()
        self.fc1 = nn.Linear(stateNum + actionNum,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.fc3 = nn.Linear(h2,h3)
        self.fc4 = nn.Linear(h3,1)
    def forward(self,state,action):
        x = torch.vstack(state,action)
        print(x.shape)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        Q = torch.relu(self.fc4(x))
        return Q

In [6]:
class Actor(nn.Module):
    def __init__(self,stateNum,h1,h2,h3,actionNum):
        super().__init__()
        self.fc1 = nn.Linear(stateNum,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.fc3 = nn.Linear(h2,h3)
        self.fc4 = nn.Linear(h3,actionNum)

    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        A = torch.tanh(self.fc4(x))
        return A

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.manual_seed(SEED)
critic        = Critic(stateNum,400,200,100,actionNum).to(device)
actor         = Actor(stateNum,400,200,100,actionNum).to(device)
critic_target = Critic(stateNum,400,200,100,actionNum).to(device)
actor_target  = Actor(stateNum,400,200,100,actionNum).to(device)
critic_target.load_state_dict(critic.state_dict())
actor_target.load_state_dict(actor.state_dict())

critic_criterion = nn.MSELoss()
# actor_criterion = 
critic_optim =  Adam(critic.parameters(),LR)
actor_optim= Adam(actor.parameters(),LR)

buffer = ExperienceReplay(BATCH_SIZE,device)

In [8]:
torch.tensor([123]).numpy()

array([123], dtype=int64)

In [9]:
for episode in range(1):
    old_observation,info =env.reset(seed=SEED)
    rewards = []
    for step in range(65):
        old_observation=torch.Tensor(old_observation).reshape(1,-1).to(device)
        action = actor(old_observation).cpu().detach().numpy().squeeze()
        old_observation=old_observation.cpu().detach().numpy().squeeze()
        new_observation,reward,terminated,truncated,info = env.step(action)
        done = terminated or truncated
        buffer.append(old_observation,action,reward,new_observation,done)
        rewards.append(reward)
        old_observation = new_observation
        if(buffer.size() >= BATCH_SIZE):
            old_state,old_action,reward,new_state,done=buffer.sample(BATCH_SIZE)
            print(reward.shape)
            y = rewards + GAMMA * critic_target(new_state,actor_target(new_state))
            y_hat = critic(old_state,actor(old_state))
            print(y_hat.shape)

torch.Size([64])


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [ ]:
torch.tensor().tolist()